In [60]:
import time
import numpy as np
import dask.array as da

# Read the grid in the text file
def load_grid_from_file(input_file):
    # Initialize an empty grid, width and height
    grid = []
    # Read positions of living cells and update the grid
    for line in input_file:
        row = line.split()
        row = list(map(int, row))
        grid.append(row)
        
    grid = np.array(grid)
    return grid

# save the grid to the text file
def save_grid_to_file(output_file, grid):
    for row in grid:
        for cell in row:
            output_file.write(str(cell)+" ")
        output_file.write('\n')

# to calculate the alive neighbors
def count_live_neighbors(grid, x, y, width, height):
    # the neighbor matrix
    neighbors = np.array([
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1),  (1, 0),  (1, 1)
    ])
    # initialize the count
    count = 0
    # loop all possible neighbors
    for dx, dy in neighbors:
        nx, ny = x + dx, y + dy
        # if its neighbor doesn't go beyond the boundary and is alive 
        if 0 <= nx < width and 0 <= ny < height and grid[ny][nx]==1:
            count += 1

    return count

# update the gird
def update_grid(grid):
    # Get height and width
    height = len(grid)
    width = len(grid[0])
    new_grid = np.array([[0] * width for _ in range(height)])

    #apply the rule
    for y in range(height):
        for x in range(width):
            live_neighbors = count_live_neighbors(grid, x, y, width, height)

            if grid[y][x] == 1:
                # Live cell
                if live_neighbors == 2 or live_neighbors == 3:
                    new_grid[y][x] = 1
            else:
                # Dead cell
                if live_neighbors == 3:
                    new_grid[y][x] = 1

    return new_grid

# run the game
def run_game(input_name, output_name, generations):
    with open(input_name, 'r') as input_file:
        grid = load_grid_from_file(input_file)
    # construct dask array
    
    
    start = time.time()
    for _ in range(generations):
        #grid = update_grid(grid)
        # Dask compute
        grid_da = da.from_array(grid, chunks=(2, 2))
        grid_da_n = grid_da.map_overlap(update_grid, depth=1, boundary="none")
        grid = grid_da_n.compute()

       # print("{} seconds elapsed for {} generations.".format(round(time.time() - start, 5), _))

    with open(output_name, 'w') as output_file:
        save_grid_to_file(output_file, grid)
    return grid


In [61]:
## main function
if __name__ == "__main__":
    input_name = "test_data/input.txt"
    output_name = "test_data/output.txt"
    generations = 100
    grid = run_game(input_name, output_name, generations)
print(grid)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 1 0]
 [0 0 1 1 0]
 [0 0 0 0 0]]


In [ ]:
new_grid = np.array([[0] * 5 for _ in range(5)])

In [ ]:
a_da_n = a_da.map_overlap(f, depth=1, boundary=0)